In [10]:
import json
from pathlib import Path
import pickle

import numpy as np

from astropy.table import Table

from stellarphot.transit_fitting.gui.gui_core import (
    exotic_arguments,
    generate_json_file_name,
    get_values_from_widget,
    exotic_settings_widget,
    populate_TOI_boxes,
)
from stellarphot.settings.fits_opener import FitsOpener

# Instructions and Overview

This notebook will do a different kind of fit to your exoplanet data. It takes longer to run than the fitting we've done so far, which is why we didn't begin with this type of fit.

In [ ]:
fits_openr = FitsOpener(
    title="Select your photometry/relative flux file", filter_pattern=["*.csv"]
)

fits_openr.file_chooser

In [ ]:
input_data_file = fits_openr.path
toi_info_file = "tess-info.pickle"

with open(toi_info_file, "rb") as f:
    tess_info = pickle.load(f)

In [ ]:
settings = exotic_settings_widget()

exotic_data_file = "data_for_exotic.csv"

In [ ]:
inp_data = Table.read(input_data_file)
inp_data = inp_data[inp_data["star_id"] == 1]

# inp_data = inp_data[inp_data['BJD'] > 2459795.75]

out_data = inp_data["BJD", "relative_flux", "relative_flux_error", "airmass"]
out_data.write(exotic_data_file, overwrite=True)

In [ ]:
settings.data_file_widget["candidate"].value = exotic_data_file
settings.data_file_widget["known"].value = exotic_data_file
populate_TOI_boxes(tess_info, settings.value_widget)

In [ ]:
settings

In [ ]:
settings_name = generate_json_file_name(settings)
contents = get_values_from_widget(settings)

In [ ]:
Path(contents["user_info"]["Directory to Save Plots"]).mkdir(exist_ok=True)

In [ ]:
with open(settings_name, "w") as f:
    json.dump(contents, f)

In [ ]:
print(
    "COPY AND PASTE THE LINE BELOW INTO AN EMPTY CELL TO RUN EXOTIC (include the exclamation point at the beginning):\n\n"
)
print(
    f'!exotic {" ".join(exotic_arguments[settings.planet_type.value])} {settings_name}'
)
print("\n\n")

In [ ]:
!exotic --override --pre TIC_402828941-2022-08-03-SI.json